# ADM Homework 3

## Search Engine

In [1]:
if True:
    %pip install -qqq numpy scipy matplotlib pandas scikit-learn nltk forex_python babel

Note: you may need to restart the kernel to use updated packages.


In [2]:
import string
import nltk
import pandas as pd
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.tokenize import word_tokenize
import re
from forex_python.converter import CurrencyRates
from babel.numbers import get_currency_symbol
from nltk.stem import *

In [3]:
nltk.download('stopwords')
nltk.download('punkt')
stemmer = PorterStemmer()
masters = pd.read_json(r"C:\Users\Jacopo\Downloads\ADM\csvjson.json")

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Jacopo\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Jacopo\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


### Preprocessing 2.0

In [4]:
# 2.0.0 Tokenize, remove stopwords and punctuation, and stem

stop_words = set(stopwords.words('english'))

def tok_punct_and_stop(text):
    # tokenize the text given
    desc_tok = word_tokenize(text)

    # remove punctuation and stopwords from the tokens
    desc_tok_nostop_nopunct = [x for x in desc_tok if x.lower() not in stop_words and x not in string.punctuation]

    # stem using porterstemmer
    stemmed_desc = [stemmer.stem(x) for x in desc_tok_nostop_nopunct]
    return stemmed_desc

# Apply the function to the description column of our dataset

masters["description"] = masters["description"].apply(tok_punct_and_stop)

In [5]:
masters

,index,courseName,universityName,facultyName,isItFullTime,description,startDate,fees,modality,duration,city,country,administration,url,
0,0,Accounting and Finance - MSc,University of Leeds,Leeds University Business School,Full time,"[busi, govern, reli, ...]",September,"UK: £18,000 (Total) International: £34,750 (To...",MSc,1 year full time,Leeds,United Kingdom,On Campus,,
1,1,"Accounting, Accountability & Financial Managem...",King’s College London,King’s Business School,Full time,"[account, account, financi, manag, msc, cours,...",September,Please see the university website for further ...,MSc,1 year FT,London,United Kingdom,On Campus,,
2,2,"Accounting, Financial Management and Digital B...",University of Reading,Henley Business School,Full time,"[embark, profession, account, career, ...]",September,Please see the university website for further ...,MSc,1 year full time,Reading,United Kingdom,On Campus,,
3,3,Addictions MSc,King’s College London,"Institute of Psychiatry, Psychology and Neuros...",Full time,"[join, us, onlin, session, prospect, ...]",September,Please see the university website for further ...,MSc,One year FT,London,United Kingdom,On Campus,,
4,4,Advanced Chemical Engineering - MSc,University of Leeds,School of Chemical and Process Engineering,Full time,"[advanc, chemic, engin, msc, leed, ...]",September,"UK: £13,750 (Total) International: £31,000 (To...",MSc,1 year full time,Leeds,United Kingdom,On Campus,,
5,,,,,,[],,,,,,,,,
6,,,,,,[],,,,,,,,,
7,,,,,,[],,,,,,,,,
8,,,,,,[],,,,,,,,,
9,,,,,,[],,,,,,,,,


In [ ]:
# 2.0.1 fee preprocessing

# function to convert all the entries to a unified currency
def convert_to_eur(amount, currency):
    c = CurrencyRates()
    eur_rate = c.get_rate(currency, 'EUR')
    return amount * eur_rate


# set up the currency conversion

# step 1: create a dictionary with the possible ways any of the 3 main currencies may be written

Currencies = {"EUR": "€, euros, EUROS, EUR, euro, eur", "USD": "$, dollar, dollars, usd",
              "GBP": "£, pound, pounds, GBP"}

# step 2: function to find numerical entries and keep surrounding elements
# step 3: convert to euros using forex library
# step 4: return the maximum value, if not specified return NaN(float)


# Apply the function on the fees column
masters["fees"] = masters["fees"].apply(one_fee)

print(masters["fees"])

In [32]:
# 2.0.1 fee preprocessing
# step 1: set up the currency conversion
# step 1.1: function to standardize currencies(most common cases), and convert to euros
def standardize_currency(input_currency):
    currency_codes = CurrencyCodes()
    # Try to get the currency code directly
    code = currency_codes.get_currency_code(input_currency)
    # If the direct lookup fails, try to get the currency code by symbol
    if not code:
        code = currency_codes.get_currency_code_by_symbol(input_currency)
    return code


def convert_to_eur(amount, currency):
    c = CurrencyRates()
    eur_rate = c.get_rate(currency, 'EUR')
    return amount * eur_rate


# step 2: function to find numerical entries and keep surrounding elements
def extract_amount_and_curr(text):
    currencies = ["usd", "$", "dollars", "dollar", "euro", "euros", "€", "gbp", "eur", "£", "pounds"]
    amount = []
    i = 0
    text_tok = tok_punct_and_stop(text)
    tex_l = []
    for entry in text_tok:
        # Find the index of the first digit
        digit_index = next((i for i, c in enumerate(entry) if c.isdigit()), None)

        if digit_index is not None:
            # Split the entry into two parts
            prefix = entry[:digit_index]
            suffix = entry[digit_index:]

            # Add the parts to the result list
            tex_l.append(prefix)
            tex_l.append(suffix)
        else:
            # If no digit is found, keep the original entry
            tex_l.append(entry)

    # parse through obtained list to find numbers and currency symbols
    """
    while i < len(tex_l):
        if tex_l[i].lower() in currencies:
            amount.append(tex_l[i])
            if tex_l[i-1].replace(',','').isnumeric():
                amount.append(tex_l[i-1].replace(',',''))
            elif tex_l[i+1].replace(',','').isnumeric():
                amount.append(tex_l[i+1].replace(',',''))
        i += 1
    return amount
    """
    while i < len(tex_l):
        if tex_l[i].lower() in currencies:
            if i - 1 >= 0 and tex_l[i-1].replace(',', '').isnumeric():
                amount.append(tex_l[i-1].replace(',', ''))
            elif i + 1 < len(tex_l) and tex_l[i+1].replace(',', '').isnumeric():
                amount.append(tex_l[i+1].replace(',', ''))
            amount.append(tex_l[i])
        i += 1
    return amount

k = "UK: 18,000£ (Total) International: £34750 (Total)"
print(extract_amount_and_curr(k))

# step 3: convert to euros using forex library



# step 4: return the maximum value, if not specified return NaN(float)


# Apply the function on the fees column
# masters["fees"] = masters["fees"].apply(one_fee)

# print(masters["fees"])

['34750', '£']


### Build a vocabulary

In [6]:
from collections import Counter
from functools import reduce

vocabulary_alt = Counter(reduce(lambda x, y: x + y, masters.description.values)).keys()
index = {}
unique_id = 1
for word in list(vocabulary_alt):
  index[unique_id] = word
  unique_id+=1
index

{1: 'busi',
 2: 'govern',
 3: 'reli',
 4: '...',
 5: 'account',
 6: 'financi',
 7: 'manag',
 8: 'msc',
 9: 'cours',
 10: 'provid',
 11: 'embark',
 12: 'profession',
 13: 'career',
 14: 'join',
 15: 'us',
 16: 'onlin',
 17: 'session',
 18: 'prospect',
 19: 'advanc',
 20: 'chemic',
 21: 'engin',
 22: 'leed'}

In [7]:
terms = pd.DataFrame(data=list(vocabulary_alt), columns=['term'])
terms['reverse'] = terms.term.apply(lambda item: list(masters.loc[masters.description.apply(lambda row: item in row)].index)) #apply instead of progress_apply
terms.head()

,term,reverse
0,busi,[0]
1,govern,[0]
2,reli,[0]
3,...,"[0, 1, 2, 3, 4]"
4,account,"[1, 2]"


In [19]:
def find_word_in_documents(word):
    filtered_rows = terms[terms['term'] == word]['reverse'].values[0]
    result_df = masters.loc[filtered_rows, ['courseName', 'universityName', 'description', 'url']]
    return result_df

word_to_search = "account"
f_n_words = find_word_in_documents(word_to_search)
f_n_words

,courseName,universityName,description,url
1,"Accounting, Accountability & Financial Managem...",King’s College London,"[account, account, financi, manag, msc, cours,...",
2,"Accounting, Financial Management and Digital B...",University of Reading,"[embark, profession, account, career, ...]",


In [11]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf = TfidfVectorizer(input='content', lowercase=False, tokenizer=lambda text: text) # , max_df=0.1
results = tfidf.fit_transform(masters.description)
result_dense = results.todense()
tfidf_data = pd.DataFrame(result_dense.tolist(), index=masters.index, columns=tfidf.get_feature_names_out())
len(tfidf_data)
tfidf_data

c:\Users\Jacopo\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\feature_extraction\text.py:525: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


,...,account,advanc,busi,career,chemic,cours,embark,engin,financi,...,leed,manag,msc,onlin,profession,prospect,provid,reli,session,us
0,0.377604,0.000000,0.000000,0.534607,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.534607,0.000000,0.000000
1,0.242641,0.612587,0.000000,0.000000,0.000000,0.000000,0.343528,0.000000,0.000000,0.343528,...,0.000000,0.343528,0.306294,0.000000,0.000000,0.000000,0.343528,0.000000,0.000000,0.000000
2,0.340861,0.430281,0.000000,0.000000,0.482588,0.000000,0.000000,0.482588,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.482588,0.000000,0.000000,0.000000,0.000000,0.000000
3,0.301206,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.426445,0.000000,0.426445,0.000000,0.000000,0.426445,0.426445
4,0.306983,0.000000,0.434624,0.000000,0.000000,0.434624,0.000000,0.000000,0.434624,0.000000,...,0.434624,0.000000,0.387516,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
5,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
6,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
7,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
8,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
9,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


In [13]:
from sklearn.metrics.pairwise import cosine_similarity
cossim_data =pd.DataFrame(cosine_similarity(tfidf_data), index=masters.index, columns=masters.index)
cossim_data

,0,1,2,3,4,5,6,7,8,9,...,20,21,22,23,24,25,26,27,28,29
0,1.000000,0.091622,0.128710,0.113737,0.115918,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.091622,1.000000,0.346291,0.073085,0.193180,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.128710,0.346291,1.000000,0.102669,0.104639,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.113737,0.073085,0.102669,1.000000,0.092465,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.115918,0.193180,0.104639,0.092465,1.000000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [24]:
def top_5_cosine_similarity_documents(word):
    word_documents = find_word_in_documents(word)
    document_indices = word_documents.index.tolist()
    cosine_scores = cossim_data.loc[document_indices].sum(axis=1)
    top_documents_indices = cosine_scores.nlargest(5).index
    top_documents_info = masters.loc[top_documents_indices, ['courseName', 'universityName', 'description', 'url']]
    return top_documents_info

word_to_search = "account"
top_document = top_5_cosine_similarity_documents(word_to_search)
top_document

,courseName,universityName,description,url
1,"Accounting, Accountability & Financial Managem...",King’s College London,"[account, account, financi, manag, msc, cours,...",
2,"Accounting, Financial Management and Digital B...",University of Reading,"[embark, profession, account, career, ...]",


In [34]:
import heap
def create_inverted_index(tfidf_data):
    inverted_index = {}
    for term_id, term in enumerate(tfidf_data.columns):
        term_info = []
        for doc_id in tfidf_data.index:
            tfidf_score = tfidf_data.at[doc_id, term]
            if tfidf_score > 0:
                term_info.append((doc_id, tfidf_score))
        inverted_index[term_id + 1] = term_info
    return inverted_index

print(inverted_index)

{1: [(0, 0.3776036208161989), (1, 0.24264064298410642), (2, 0.3408609831319905), (3, 0.3012060645088279), (4, 0.30698346482717465)], 2: [(1, 0.6125872523158268), (2, 0.4302805385579505)], 3: [(4, 0.43462416830508027)], 4: [(0, 0.5346074901415979)], 5: [(2, 0.4825876253132941)], 6: [(4, 0.43462416830508027)], 7: [(1, 0.3435282343736251)], 8: [(2, 0.4825876253132941)], 9: [(4, 0.43462416830508027)], 10: [(1, 0.3435282343736251)], 11: [(0, 0.5346074901415979)], 12: [(3, 0.42644458179301653)], 13: [(4, 0.43462416830508027)], 14: [(1, 0.3435282343736251)], 15: [(1, 0.3062936261579134), (4, 0.38751578241817713)], 16: [(3, 0.42644458179301653)], 17: [(2, 0.4825876253132941)], 18: [(3, 0.42644458179301653)], 19: [(1, 0.3435282343736251)], 20: [(0, 0.5346074901415979)], 21: [(3, 0.42644458179301653)], 22: [(3, 0.42644458179301653)]}


In [35]:
def find_related_documents(word, masters, tfidf_data, cossim_data, inverted_index):
    term_id = terms[terms['term'] == word].index[0] + 1
    term_info = inverted_index.get(term_id, [])
    if not term_info:
        print(f"No documents found for the word '{word}'.")
        return
    term_info.sort(key=lambda x: x[1], reverse=True)
    top_documents = []
    heap = []
    for doc_id, tfidf_score in term_info:
        similarity_score = cossim_data.at[masters.index[doc_id], masters.index[doc_id]]
        heapq.heappush(heap, (similarity_score, doc_id))
        if len(heap) > 5:
            heapq.heappop(heap)
    top_documents = sorted(heap, key=lambda x: x[0], reverse=True)
    
    # Display the results as a table
    result_table = pd.DataFrame(columns=['courseName', 'universityName', 'description', 'url', 'Similarity'])
    
    for similarity, doc_id in top_documents:
        row_data = masters.loc[masters.index[doc_id], ['courseName', 'universityName', 'description', 'url']].tolist()
        row_data.append(similarity)
        result_table = pd.concat([result_table, pd.DataFrame([row_data], columns=result_table.columns)], ignore_index=True)
    return result_table

In [37]:
inverted_index = create_inverted_index(tfidf_data)
word_to_search = "account"
related_documents = find_related_documents(word_to_search, masters, tfidf_data, cossim_data, inverted_index)
related_documents

C:\Users\Jacopo\AppData\Local\Temp\ipykernel_6496\2196086629.py:34: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  result_table = pd.concat([result_table, pd.DataFrame([row_data], columns=result_table.columns)], ignore_index=True)


,courseName,universityName,description,url,Similarity
0,"Accounting, Financial Management and Digital B...",University of Reading,"[embark, profession, account, career, ...]",,1.0
